In [1]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
import pickle

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
data = pd.read_csv("feature.csv")

In [5]:
data.head()

,Unnamed: 0,File,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,has_ip,...,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,0,spam_dataset,31,0.064516,0,0,-4.708307,True,False,0,...,0,0,0,0,0.000000,0,2,0.000000,False,81
1,1,spam_dataset,64790,0.306050,0,0,-4.576882,True,False,0,...,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,2,spam_dataset,757,0.195509,0,0,-4.205536,True,False,0,...,0,1,0,0,0.000000,0,148,0.000000,False,46
3,3,spam_dataset,266310,0.230915,0,0,-4.550931,True,False,0,...,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,4,spam_dataset,40,0.200000,0,0,-4.689336,True,False,0,...,2,1,0,0,0.000000,0,8,0.000000,False,83


In [6]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [7]:
data = data.drop(columns = "File")

In [8]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [12]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp1cw_fots.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp1cw_fots.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
224/224 [==============================] - 12s 4ms/step - loss: 1.4192 - accuracy: 0.3614 - f1_m: 0.1632 - precision_m: 0.5647 - recall_m: 0.0977 - val_loss: 1.1478 - val_accuracy: 0.5310 - val_f1_m: 0.4330 - val_precision_m: 0.7517 - val_recall_m: 0.3049
Epoch 2/50
224/224 [==============================] - 0s 2ms/step - loss: 1.0882 - accuracy: 0.5574 - f1_m: 0.4769 - precision_m:

224/224 [==============================] - 0s 2ms/step - loss: 0.4687 - accuracy: 0.8154 - f1_m: 0.8144 - precision_m: 0.8709 - recall_m: 0.7653 - val_loss: 0.4968 - val_accuracy: 0.8150 - val_f1_m: 0.8085 - val_precision_m: 0.8653 - val_recall_m: 0.7590
Epoch 29/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.8163 - f1_m: 0.8137 - precision_m: 0.8714 - recall_m: 0.7636 - val_loss: 0.4950 - val_accuracy: 0.8118 - val_f1_m: 0.8057 - val_precision_m: 0.8630 - val_recall_m: 0.7559
Epoch 30/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4567 - accuracy: 0.8240 - f1_m: 0.8177 - precision_m: 0.8741 - recall_m: 0.7686 - val_loss: 0.4969 - val_accuracy: 0.8114 - val_f1_m: 0.8116 - val_precision_m: 0.8619 - val_recall_m: 0.7670
Epoch 31/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.8226 - f1_m: 0.8210 - precision_m: 0.8728 - recall_m: 0.7755 - val_loss: 0.4786 - val_accuracy: 0.8135 - val_f1_m: 

In [13]:
y_pred = model.predict(X_test)

In [14]:
predicted = np.argmax(y_pred, axis=1)

In [15]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8322452871708355


In [16]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.88      0.97      0.92      2735
  Defacement       0.84      0.67      0.75      2499
     Malware       0.82      0.87      0.84      2834
    Phishing       0.73      0.78      0.75      2477
        Spam       0.89      0.85      0.87      3088

    accuracy                           0.83     13633
   macro avg       0.83      0.83      0.83     13633
weighted avg       0.83      0.83      0.83     13633



In [17]:
#os.chdir("../")
#os.chdir("models")
model.save("Model_v1.h5")
np.save('lblenc.npy', encoder.classes_)
scalerfile = 'scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [20]:
scalerfile = 'scaler.sav'
scaler1 = pickle.load(open(scalerfile, 'rb'))

In [23]:
pip install sklearn==0.24.2

ERROR: Could not find a version that satisfies the requirement sklearn==0.24.2 (from versions: 0.0)
ERROR: No matching distribution found for sklearn==0.24.2
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 2.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
Note: you may need to restart the kernel to use updated packages.
